In [71]:
import numpy as np
import pandas as pd
from osgeo import gdal
from gdalconst import *
#import osr
import matplotlib.pyplot as plt
%matplotlib qt




In [72]:
#Read a raster
Test_data = ("/Users/bmelosh/VagrantBoxes/LSDTopoTools/Topographic_projects/Test_data/")
Test = gdal.Open(Test_data + "/Mandakini.bil")#Get raster data
M_DEM = Test.ReadAsArray()

#Read stream data
name = "Mandakini_fullProfileMC_forced_0.45_3_1258909000_10_80_281_for_Arc.csv" #Set file name
path = "/Users/bmelosh/VagrantBoxes/LSDTopoTools/Topographic_projects/Test_data/" #Set path to data
#for csv files
data = np.genfromtxt((path+name), delimiter=',', skip_header=1, names=['id', 'x', 'y', 'chan_number', 'reciever_chan','node_on_reciever_chan', 'node', 'row','column', 'flow_distance', 'chi', 'elevation', 'drainage_area', 'n_data_points', 'm_mean','m_st_dev', 'm_std_err', 'b_mean', 'b_st_dev', 'b_std_err', 'DW_mean', 'DW_st_dev', 'DW_std_err', 'fitted_elev_mean', 'fitted_elev_stdev', 'fitted_elev_std_err'])
#for tree files
#data = np.genfromtxt((path+name), delimiter=' ', skip_header=1, names=['chan_number', 'reciever_chan','node_on_reciever_chan', 'node', 'row','column', 'flow_distance', 'chi', 'elevation', 'drainage_area', 'n_data_points', 'm_mean','m_st_dev', 'm_std_err', 'b_mean', 'b_st_dev', 'b_std_err', 'DW_mean', 'DW_st_dev', 'DW_std_err', 'fitted_elev_mean', 'fitted_elev_stdev', 'fitted_elev_std_err'])


In [73]:



def plot_coord(Test):

    (upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = Test.GetGeoTransform()
    dx = x_size
    dy = y_size
    nx, ny = Test.RasterXSize, Test.RasterYSize  # Size of the original raster

    xllcenter = upper_left_x + dx/2  # x coordinate center of lower left pxl
    yllcenter = upper_left_y - dx/2 # y coordinate center of lower left pxl   - (ny-1)*dx 

    #Create arrays of the x and y coordinates of each pixel (the axes)
    xcoordinates = [x*dx + xllcenter for x in range(nx)]
    ycoordinates = [y*dy + yllcenter for y in range(ny)]

    #Create 2 2d grids describing x and y coordinates
    X,Y = np.meshgrid(xcoordinates, ycoordinates) 


    fig = plt.contourf(X, Y, M_DEM, levels=np.linspace(np.amin(M_DEM[M_DEM > 0]),np.amax(M_DEM), 50))

    return X, Y


In [79]:
(upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = Test.GetGeoTransform()
dx = x_size
dy = y_size
nx, ny = Test.RasterXSize, Test.RasterYSize  # Size of the original raster

xllcenter = upper_left_x + dx/2  # x coordinate center of lower left pxl
yllcenter = upper_left_y - dx/2 # y coordinate center of lower left pxl   - (ny-1)*dx 

#Create arrays of the x and y coordinates of each pixel (the axes)
xcoordinates = [x*dx + xllcenter for x in range(nx)]
ycoordinates = [y*dy + yllcenter for y in range(ny)]

#Create 2 2d grids describing x and y coordinates
X,Y = np.meshgrid(xcoordinates, ycoordinates) 


fig = plt.contourf(X, Y, M_DEM, levels=np.linspace(np.amin(M_DEM[M_DEM > 0]),np.amax(M_DEM), 50))
    

In [75]:
[X, Y] = plot_coord(Test)

In [76]:
cm = plt.cm.gist_earth


fig, ax = plt.subplots(3, figsize=(10,10), sharex=False) #define 2 subfigures

ax[0].contourf(X, Y, M_DEM, levels=np.linspace(np.amin(M_DEM[M_DEM > 0]),np.amax(M_DEM), 50))
ax[0].plot(data['x'], data['y'], 'ro',  markersize=2)
#ax[0].imshow(fA, interpolation='nearest', vmin=0, cmap=plt.cm.gray) 
ax[0].set_aspect('equal')
ax[0].set_title('Terrain map with stream channel')
ax[0].set_ylabel('Northing')
ax[0].set_xlabel('Easting')


ax[1].scatter(data['flow_distance'], data['elevation'], c=data['elevation'], cmap=cm)
ax[1].set_title('Stream Map')
ax[1].set_ylabel('Elevation (m)')
ax[1].set_xlabel('Distance (m)')


ax[2].scatter(data['chi'], data['elevation'], c=data['elevation'], cmap=plt.get_cmap("jet"))
ax[2].text(3, 2500, 'm/n = 0.45')
ax[2].set_ylabel('Elevation (m)')
ax[2].set_xlabel('Chi (X)')


plt.show()

In [82]:
Y

array([[ 3410028.75864564,  3410028.75864564,  3410028.75864564, ...,
         3410028.75864564,  3410028.75864564,  3410028.75864564],
       [ 3409939.94451675,  3409939.94451675,  3409939.94451675, ...,
         3409939.94451675,  3409939.94451675,  3409939.94451675],
       [ 3409851.13038785,  3409851.13038785,  3409851.13038785, ...,
         3409851.13038785,  3409851.13038785,  3409851.13038785],
       ..., 
       [ 3352743.64550734,  3352743.64550734,  3352743.64550734, ...,
         3352743.64550734,  3352743.64550734,  3352743.64550734],
       [ 3352654.83137845,  3352654.83137845,  3352654.83137845, ...,
         3352654.83137845,  3352654.83137845,  3352654.83137845],
       [ 3352566.01724955,  3352566.01724955,  3352566.01724955, ...,
         3352566.01724955,  3352566.01724955,  3352566.01724955]])

In [77]:
#Import bokeh 
from bokeh.plotting import *
from bokeh.models import ColumnDataSource, Label
from bokeh.layouts import layout

#package in this working directory
from utils.colormap import RGBAColorMapper
import netCDF4
from bokeh.palettes import RdBu11



In [89]:
#My bokeh figure
mn = " (m/n = 0.45)"

# output to static HTML file
output_file("linked_brushing.html")

# NEW: create a column data source for the plots to share
stream_source = ColumnDataSource(data=dict(x0=data['flow_distance'], x1=data['chi'], y=data['elevation']))

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=350, height=350, title='Stream Profile')
left.circle('x0', 'y', source=stream_source)
#left.title.text = "Glucose Range"
#left.xaxis.axis_label = 'Flow Distance (m)'
#left.yaxis.axis_label = 'Elevation (m)'

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=350, height=350, title='Chi Profile'+ mn)
right.circle('x1', 'y', source=stream_source)#, legend = "m/n = 0.45")
#right.xaxis.axis_label = 'Chi (X)'
#right.yaxis.axis_label = 'Elevation (m)'

dem = figure(tools=TOOLS, width=700, height=350, title='DEM')
#dem.image(X[0], x=0, y=0, dw=10, dh=10, palette="Spectral11")
q.image_rgba(image=[X], 
            x=[-180], y=[-90],
    dw=[360], dh=[180], name='image'
)

# put the subplots in a gridplot
p = layout([[left, right], [dem]])

# must give a vector of image data for image parameter
#p.image(image=[dem], x=0, y=0, dw=10, dh=10, palette="Spectral11")



# l = layout([
#   [bollinger],
#   [sliders, plot],
#   [p1, p2, p3],
# ], sizing_mode='stretch_both')


# show the results
show(p)

In [15]:
#attempting to plot raster

import numpy as np

from bokeh.plotting import figure, show, output_file


xx, yy = np.meshgrid(data['x'], data['y'])
d = data['elevation']


p = figure(x_range=(0,100), y_range=(0,100))#(min(data['x']),max(data['x'])), y_range=(min(data['y']),max(data['y'])))
# must give a vector of image data for image parameter
p.image(image=[d], x=0, y=0, dw=10, dh=10, palette="Spectral11")

output_file("image.html", title="image.py example")

show(p)  # open a b

In [69]:

from bokeh.plotting import figure, output_notebook, show
data = netCDF4.Dataset('Land_and_Ocean_LatLong1.nc')
t = data.variables['temperature']

colormap = RGBAColorMapper(-6, 6, RdBu11)
# Set output option
output_notebook()


# Write a function ``get_slice(temperature_variable, year, month)`` that given a temperature variable, year and month
# returns an array of rgba colors. (Hint: Use RGBAColorMapper.color()). 
def get_slice(t, year, month):
    i = (year - 1850)*12 + month - 1
    return colormap.color(t[i, :, :])

# Get image data for a given year and month using the ``get_slice`` function
image = get_slice(t, 1950, 1)




# Create plot
# Tip: Make sure to set x_range and y_range appropriately
p =  figure(width=900, height=500, x_axis_type=None, y_axis_type=None, x_range=[-180,180], y_range=[-90,90])

p.image_rgba(
    image=[image],
    x=[-180], y=[-90],
    dw=[360], dh=[180], name='image'
)

q = figure(width=900, height=500, x_axis_type=None, y_axis_type=None, x_range=[-180,180], y_range=[-90,90])
q.image_rgba(image=[dem], 
            x=[-180], y=[-90],
    dw=[360], dh=[180], name='image'
)


# Show plot
show(q)

Loading BokehJS ...

/Users/bmelosh/anaconda2/envs/geospatial/lib/python2.7/site-packages/ipykernel/__main__.py:15: RuntimeWarning: invalid value encountered in less
/Users/bmelosh/anaconda2/envs/geospatial/lib/python2.7/site-packages/ipykernel/__main__.py:15: RuntimeWarning: invalid value encountered in greater


In [67]:
type(image)


numpy.ma.core.MaskedArray

In [13]:

# Initialize the colormap with a bokeh palette and the low and high values for the scale.
#from bokeh.palettes import RdBu11
#colormap = RGBAColorMapper(-6, 6, RdBu11)

p =  figure(width=900, height=500, x_axis_type=None, y_axis_type=None, x_range=[-180,180], y_range=[-90,90])

p.image_rgba(
    image=[M_DEM],
    x=[-180], y=[-90],
    dw=[360], dh=[180], name='image'
)



GlyphRenderer(id='deef424b-3843-41e8-9044-391385a46509', ...)

In [20]:
show(p)